In [1]:
!pip install --quiet  datasets #to access race dataset
!pip install --quiet pyarrow   #to deal with parquet files for saving dataset if required
!pip install --quiet  tqdm     #for progress bars
!pip install --quiet transformers # for t5 model
!pip install --quiet tokenizers  #tokenizers from HuggingFace
!pip install --quiet sentencepiece #subword tokenizer used by T5
!pip install --quiet pytorch-lightning # pytorch wrapper
!pip install --quiet torchtext # text utilities
!pip install SentencePiece library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 49.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for library: filename=Library-0.0.0-py3-none-any.whl size=2054 sha256=46c2c935ace68b297bd54e074f026681e77a2029f7e78c220b43240468cd4404
  Stored in directory: /root/.cache/pip/wheels/e0/71/7d/b0e29b944e43374597cd4e3b88c85197001c9bfcd5dce191f4
Successfully built library


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#imports
import pandas as pd
import torch
from tqdm import tqdm
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from pprint import pprint
import copy

In [4]:
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

In [8]:
trained_model_path1 = '/content/drive/MyDrive/t5_trained_model2'
trained_tokenizer1 = '/content/drive/MyDrive/t5_tokenizer2'
device = 'cpu'
model1 = T5ForConditionalGeneration.from_pretrained(trained_model_path1)
tokenizer1 = T5Tokenizer.from_pretrained(trained_tokenizer1)

In [13]:
def get_question1(sentence,mdl,tknizer):

  ''' function to generate questions. Takes a sentence,answer,
      model and tokenizer
  '''

  text = "context: {}".format(sentence)
  #print (text)
  max_len = 256
  encoding = tknizer.encode_plus(text,max_length=max_len, pad_to_max_length=False,truncation=True, return_tensors="pt")

  input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

  outs = mdl.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=5,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  max_length=72)
  #print("outputs")
  #print(outs)
  for beam_output in outs:
    sent = tknizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    #print(sent)

  dec = [tknizer.decode(ids,skip_special_tokens=True) for ids in outs]
  #print("decs")
  #print(dec)

  Question = dec[0]
  #Question= Question.strip()
  #Answer= Answer.strip()
  #print(Answer)
  index = Question.find("answer:")

  # Extract the question and answer based on the position
  question = Question[10:index].strip()
  answer = Question[index + len("answer:"):].strip()
  #print("Question1:", question)
  #print("Answer1:", answer)

  return  question,answer

In [14]:
context = "The Sino-Indian War also known as the China–India War, Indo-China War, Indo-China War of 1962 or Sino-Indian War of 1962, was a military conflict between China and India that took place from October to November 1962. It was a military escalation of the Sino-Indian border dispute. Fighting occurred along India's border with China, in India's North-East Frontier Agency east of Bhutan, and in Aksai Chin west of Nepal."

print("context: ",context)
ques,answer = get_question1(context,model1,tokenizer1)
print ("question: ",ques)
print ("answer: ",answer)

context:  The Sino-Indian War also known as the China–India War, Indo-China War, Indo-China War of 1962 or Sino-Indian War of 1962, was a military conflict between China and India that took place from October to November 1962. It was a military escalation of the Sino-Indian border dispute. Fighting occurred along India's border with China, in India's North-East Frontier Agency east of Bhutan, and in Aksai Chin west of Nepal.
question:  What was the military escalation of the Sino-Indian border dispute between China and India?
answer:  The conflict occurred along India's border with China, in India’s North-East Frontier Agency east of Bhutan, and in Aksai Chin west of Nepal.


Word Overlap Evaluator Scores

In [15]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=ea13373b93c1cea45f4ce8393e97a976b540d1ba43683e7ced92a9c2647e8c09
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [18]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [20]:
from datasets import load_dataset

dataset = load_dataset("race",'middle')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/1436 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/25421 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1436 [00:00<?, ? examples/s]

In [32]:
test_dataset = dataset['test']

# Access the questions from the "test" split
questions_test = test_dataset['question']

# Print the first few questions
for question in questions_test[:5]:
    print(question)

In which country is the prom called a "formal"?
Why is the prom important in the students' lives?
What is the passage mainly about?
The first kite was born when   _  .
Kites are made of the following materials EXCEPT   _   according to the passage.


In [42]:
def generate_d_fake(context, model, tokenizer, num_sentences=1):
    d_fake_sentences = []
    for _ in range(num_sentences):
        d_fake, _ = get_question1(context, model, tokenizer)
        d_fake_sentences.append(d_fake)
    return d_fake_sentences


In [25]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=eed8bbc357d774efd5bb49fef782c694105b62b778aaaa92062169d46c76473c
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [26]:
!pip install sent2vec

In [43]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
from rouge_score import rouge_scorer
import nltk
import nltk.translate.meteor_score
from sent2vec.vectorizer import Vectorizer
from scipy.spatial.distance import cosine
import numpy as np

# ... (your existing code)

# Access D_real from questions in the test part of Race Dataset
d_real_sentences = test_dataset['question']

# Generate D_fake sentences
d_fake_sentences = generate_d_fake(context, model1, tokenizer1)

# Initialize an empty list eval_scores.
eval_scores = []

# Initialize the Sent2Vec vectorizer
vectorizer = Vectorizer()

# for each D-fake sentence in List of D-fake sentences do
for d_fake in d_fake_sentences:
    # Initialize an empty list similar_real_positions.
    similar_real_positions = []

    # Calculate sentence embeddings for D-fake sentence using the vectorizer
    vectorizer.run([d_fake])
    d_fake_embedding = vectorizer.vectors[0]

    # for each D-real sentence in List of D-real sentences do
    for position, d_real in enumerate(d_real_sentences):
        # Calculate the cosine distance between the embeddings of D-fake and D-real sentences.
        vectorizer.run([d_real])
        d_real_embedding = vectorizer.vectors[0]
        cosine_distance = cosine(d_fake_embedding, d_real_embedding)

        # Append the distance and the position of D-real sentence to similar_real_positions.
        similar_real_positions.append((cosine_distance, position))

    # Sort similar_real_positions based on increasing cosine distances.
    similar_real_positions.sort()

    # Select the positions of the top 10 D-real sentences.
    selected_positions = [pos for _, pos in similar_real_positions[:10]]

    # Initialize a list of D-real sentences using the selected positions.
    selected_d_real_sentences = [d_real_sentences[pos] for pos in selected_positions]

    # Initialize a list of evaluation scores for the current D-fake sentence.
    evaluation_scores = []

    # for each selected D-real sentence do
    for selected_d_real in selected_d_real_sentences:
        # Calculate BLEU Score
        bleu_score = sentence_bleu([selected_d_real], d_fake, weights=(0.2, 0.2, 0.2, 0.2))

        # Calculate ROUGE Score
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        rouge_scores = scorer.score(selected_d_real, d_fake)

        # Calculate Meteor Score
        meteor_score = nltk.translate.meteor_score.meteor_score([word_tokenize(selected_d_real)], word_tokenize(d_fake))

        # Append the scores to the list of evaluation scores.
        evaluation_scores.append({
            'bleu_score': bleu_score,
            'rouge_scores': rouge_scores,
            'meteor_score': meteor_score
        })

    # Calculate the average of the evaluation scores for the current D-fake sentence.
    average_scores = {
        key: np.mean([score[key] for score in evaluation_scores]) if isinstance(evaluation_scores[0][key], (float, int)) else evaluation_scores[0][key]
        for key in evaluation_scores[0].keys()
    }
    # Convert the scores to float if they are not already
    average_scores = {key: float(value) if isinstance(value, (float, int)) else value for key, value in average_scores.items()}

    # Append the average evaluation scores to eval_scores.
    eval_scores.append(average_scores)

# Return eval_scores.
print("Evaluation Scores:", eval_scores)


Initializing Bert distilbert-base-uncased
Vectorization done on cuda
Evaluation Scores: [{'bleu_score': 0.19534744109441068, 'rouge_scores': {'rouge1': Score(precision=0.06666666666666667, recall=0.1111111111111111, fmeasure=0.08333333333333334), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.06666666666666667, recall=0.1111111111111111, fmeasure=0.08333333333333334)}, 'meteor_score': 0.1104218571816347}]


Bert Score

In [46]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
from rouge_score import rouge_scorer
import nltk
import nltk.translate.meteor_score
from sent2vec.vectorizer import Vectorizer
from scipy.spatial.distance import cosine
import numpy as np
from transformers import BertTokenizer, BertModel
import torch

# ... (your existing code)

# Initialize the BERT tokenizer and model
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Initialize an empty list bertscores.
bert_scores = []

# for each D-fake sentence in List of D-fake sentences do
for d_fake in d_fake_sentences:
    # Initialize an empty list bertsimilarities.
    bert_similarities = []

    # Calculate BERT embeddings for D-fake sentence using the BERT model.
    inputs_fake = bert_tokenizer(d_fake, return_tensors="pt", truncation=True)
    outputs_fake = bert_model(**inputs_fake)
    embeddings_fake = outputs_fake.last_hidden_state.mean(dim=1).detach().numpy().flatten()  # Reshape to 1-D

    # for each D-real sentence obtained from sent2vec model in List of D-real sentences do
    for d_real in d_real_sentences:
        # Calculate BERT embeddings for D-real sentence using the BERT model.
        inputs_real = bert_tokenizer(d_real, return_tensors="pt", truncation=True)
        outputs_real = bert_model(**inputs_real)
        embeddings_real = outputs_real.last_hidden_state.mean(dim=1).detach().numpy().flatten()  # Reshape to 1-D

        # Calculate cosine similarity between BERT embeddings of D-fake and D-real sentences.
        similarity = 1 - cosine(embeddings_fake, embeddings_real)

        # Append the cosine similarity to bertsimilarities.
        bert_similarities.append(similarity)

    # Calculate the average cosine similarity (BERT score).
    average_similarity = np.mean(bert_similarities)

    # Append the average cosine similarity (BERT score) to bertscores.
    bert_scores.append(average_similarity)

# Return bertscores.
print("BERT Scores:", bert_scores)


BERT Scores: [0.5390713082465621]
